<a href="https://colab.research.google.com/github/pavankonam/Cognizant/blob/main/hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [18]:
import pandas as pd
import numpy as np

Loading Data

In [20]:
li=pd.read_excel("/content/LifeUniverseAndEverything.xlsx")
li.head()

,Life,O2,O3,H2O,N2,H2,Sunlight,Aliens,Plants
0,5,6,38,48,25,38,0,1,1
1,4,0,4,2,6,13,0,1,0
2,4,11,3,78,30,15,0,0,1
3,0,22,30,68,28,23,0,1,0
4,10,20,23,56,20,27,1,0,1


Declaring a and b variables

In [21]:
a = li.drop('Life', axis=1)
b = li['Life']
li.dropna(inplace=True)
li.shape

(200, 9)

Implementing Grid Search

In [22]:
from sklearn.pipeline import Pipeline #sklearn==0.23.2
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from lightgbm import LGBMClassifier
tuples = list()
tuples.append((Pipeline([
        ('scaler', StandardScaler()),
    ]), ["O2","O3","H2O","N2","H2"]))


preprocess = make_column_transformer(*tuples)

pipe = Pipeline([
    ('preprocess', preprocess),
    ('classifier', LGBMClassifier())
])
from sklearn.model_selection import GridSearchCV
param_grid = {
    "classifier__learning_rate": [0.0001, 0.0005, 0.001, 0.01, 0.1],
    "classifier__n_estimators": [100, 300, 600, 800, 1000],
    "classifier__max_depth": [ 4, 20, 100, 250, 400] 
}

reg_grid = GridSearchCV(pipe,
                        param_grid=param_grid,
                        cv=5,
                        n_jobs=8, 
                        scoring='roc_auc'
                       )

model_grid = reg_grid.fit(a, b)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  category=UserWarning,


In [23]:
param = 'param_classifier__learning_rate'

grid = model_grid.cv_results_[param]
grid

masked_array(data=[0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
                   0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
                   0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
                   0.0001, 0.0001, 0.0001, 0.0001, 0.0005, 0.0005, 0.0005,
                   0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005,
                   0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005,
                   0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005,
                   0.0005, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
                   0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
                   0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
                   0.001, 0.001, 0.001, 0.01, 0.01, 0.01, 0.01, 0.01,
                   0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
                   0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
                   0.01, 0.01, 0.1, 0.1,

Implementing Random

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, randint

n_iter = 70

param_grid = {
    "classifier__learning_rate": loguniform(1e-4, 0.1),
    "classifier__n_estimators": randint(100,1000),
    "classifier__max_depth": randint(4, 400) 
}

reg_rand = RandomizedSearchCV(pipe,
                         param_distributions=param_grid,
                         n_iter=n_iter,
                         cv=5,
                         n_jobs=8,
                         scoring='roc_auc',
                         random_state=123)

model_rand = reg_rand.fit(a, b)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  category=UserWarning,


Implementing Bayes

In [25]:
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 3.5 MB/s 


In [26]:
from skopt import BayesSearchCV

# Bayesian
n_iter = 70

param_grid = {
    "classifier__learning_rate": (0.0001, 0.1, "log-uniform"),
    "classifier__n_estimators": (100,  1000) ,
    "classifier__max_depth": (4, 400) 
}

reg_bay = BayesSearchCV(estimator=pipe,
                    search_spaces=param_grid,
                    n_iter=n_iter,
                    cv=5,
                    n_jobs=8,
                    random_state=123)

model_bay = reg_bay.fit(a, b)